# Do logistic regression on Snippet dataset

In [ ]:
# Load packages
import numpy as np
from matplotlib import pyplot as plt
from sklearn import decomposition as dc
from sklearn import linear_model as lm

In [ ]:
# Load data
snippets = np.load('../../data/snippets.npy')
targets = np.load('../../data/targets.npy')

# Remove time
snippets = snippets[:, :, :-1]

In [ ]:
y = targets
ind_anomaly = np.where(y == 1)[0]
ind_no_anomaly = np.array_split(np.where(y == 0)[0], 9)
X = snippets.reshape(snippets.shape[0], np.prod(snippets.shape[1:]))
X_mean = np.mean(X, 1, keepdims=True)
X_std = np.std(X, 1, keepdims=True)
X_norm = (X - X_mean) / X_std

In [ ]:
# Do PCA
pca = dc.PCA(n_components=3)
pca.fit(X_norm)
X_trans = pca.transform(X_norm)

In [ ]:
# Fit logistic regression model
l = 2
for i in range(9):
    ind_fold = np.concatenate((np.array([ind_anomaly[i]]), ind_no_anomaly[i]))
    X_fold_ref = np.delete(X_trans, ind_fold, 0)[::100, :]
    X_fold_train = get_x_tilde(evaluate_basis_functions(l, np.delete(X_trans, ind_fold, 0), X_fold_ref))
    y_fold_train = np.delete(y, ind_fold, 0)
    X_fold_test = get_x_tilde(evaluate_basis_functions(l, X_trans[ind_fold, :], X_fold_ref))
    y_fold_test = y[ind_fold]
    log_reg = lm.LogisticRegression(max_iter=10000, class_weight='balanced')
    log_reg.fit(X_fold_train, y_fold_train)
    y_fold_pred = log_reg.predict(X_fold_test)
    print(np.mean(y_fold_test == y_fold_pred))
    ind_a = np.where(y_fold_test == 1)[0]
    print(y_fold_pred[ind_a])
    y_fold_pred = log_reg.predict(X_fold_train)
    #ind_a = np.where(y_fold_pred == 0)[0]
    #ind_na = np.where(y_fold_pred == 1)[0]
    #plt.figure(figsize=(15, 10))
    #plt.plot(np.delete(X_trans, ind_fold, 0)[ind_na, 0], np.delete(X_trans, ind_fold, 0)[ind_na, 1], '.')
    #plt.plot(np.delete(X_trans, ind_fold, 0)[ind_a, 0], np.delete(X_trans, ind_fold, 0)[ind_a, 1], 'r.')

In [ ]:
xx, yy = np.meshgrid(np.linspace(-22, 45), np.linspace(-25, 30))

In [ ]:
# Do PCA
pca = dc.PCA(n_components=20)
pca.fit(X_norm)
X_trans = pca.transform(X_norm)

In [ ]:
ind0 = 0
ind1 = 1
plt.plot(X_trans[:, ind0], X_trans[:, ind1], '.')
plt.plot(X_trans[ind_anomaly, ind0], X_trans[ind_anomaly, ind1], 'r.')

In [ ]:
np.savetxt('y.csv', y)

In [ ]:
np.concatenate([np.arange(1), np.arange(10)])

In [ ]:
xx.shape